<a href="https://colab.research.google.com/github/SiweiLuo/DL_with_Python/blob/master/CV_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import string 
import keras
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils
from keras.datasets import imdb

from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import text, sequence

from google.colab import drive

#import StringIO
import time
import sys

In [52]:
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
questions = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/CareerVillage/questions.csv')
#print('questions = ',questions)
answers = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/CareerVillage/answers.csv')
#print('answers = ',answers)

In [58]:
answers_body = answers.answers_body
questions_body = questions.questions_body
#answers_body = list(answers_body)
#len(answers_body)
#len(answers_body[2])
len(questions_body)

23931

In [0]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' + '\n' 
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text

    data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
    return data

In [0]:
x_train = preprocess(answers_body)
q_shape = preprocess(questions_body)

In [0]:
len(x_train[2])

3036

In [0]:
len(x_train[2])

3036

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(x_train)+list(q_shape))

In [0]:
x_train = tokenizer.texts_to_sequences(x_train)
q_shape = tokenizer.texts_to_sequences(q_shape)

In [0]:
x_train[0]

In [0]:
question_len = 40 

In [0]:
x_train = sequence.pad_sequences(x_train,maxlen=500)
q_shape = sequence.pad_sequences(q_shape,maxlen=question_len)

In [0]:
x_train[2]

In [0]:
q_shape[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,   25,   11,    5, 2449,  302,   25,   11,
          5, 2449,  302,  800,   29,  653, 2665], dtype=int32)

In [0]:
import keras 
from keras import layers
from keras import backend as K 
from keras.models import Model
import numpy as np 
from keras import Input


In [0]:
batch_size =16
latent_dim = 2 
question_shape = (40,1)
input_qus = keras.Input(shape=question_shape)
x = layers.LSTM(32,return_sequences=True,input_shape=(40,))(input_qus)
x = layers.LSTM((32))(x)
#x = layers.LSTM((32))(x)
shape_before_flattening = K.int_shape(x)

#x = layers.Flatten()(x)
x = layers.Dense(32,activation='softmax')(x)

z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

que_encoder = Model(input_qus,z_mean)



In [0]:
answer_shape = (40,1)
input_ans = keras.Input(shape=answer_shape)
x = layers.LSTM(64,return_sequences=True,input_shape=(40,))(input_ans)
x = layers.LSTM((64))(x)
#x = layers.LSTM((64))(x)
x = layers.Dense(32,activation='softmax')(x)

z_mean_ans = layers.Dense(latent_dim)(x)
z_log_var_ans = layers.Dense(latent_dim)(x)

ans_encoder = Model(input_ans,z_mean_ans)


In [0]:
#concatenated = layers.concatenate([z_mean,z_mean_ans],axis=-1)
concatenated = (z_mean-z_mean_ans)**2
combined = layers.Dense(1,input_shape = (1,))(concatenated)

correlation = Model([input_qus,input_ans])

correlation.compile(optimizer = 'rmsprop',loss=None,metrics=['acc'])


In [0]:
decoder = Sequential()

decoder.Add(layers.Dense(32,activation='relu',input_shape=(2,)))
decoder.Add(layers.Dense(32,activation='relu'))
decoder.Add(layers.Dense(32,activation='softmax'))

decoder.compile(optimizer='rmsprop',loss='categroical_crossentropy',metrics=['acc'])


In [0]:
decoder.fit()

In [14]:
shape_before_flattening 

(None, 32)

In [0]:
def sampling(args):
  z_mean, z_log_var = args
  epsilon = K.random_normal(shape=(K.shape(z_mean)[0],latent_dim),mean=0.,stddev=1.)
  return z_mean + K.exp(z_log_var)*epsilon

z = layers.Lambda(sampling)([z_mean,z_log_var])

In [0]:
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]),
                activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
#x = layers.LSTM(32)(x)
#x = layers.Dense(32,activation='softmax')(x)

#decoder = Model(decoder_input,x)
#z_decoded = decoder(z)

In [7]:
x.shape

TensorShape([Dimension(None), Dimension(32)])

In [8]:
class CustomVariationLayer(keras.layers.Layer):
  def vae_loss(self,x,z_decoded):
    #x = K.flatten(x)
    #z_decoded = K.f
    z_decoded = z_decoded
    xent_loss = keras.metrics.binary_crossentropy(x,z_decoded)
    kl_loss = -5e-4 * K.mean(1+z_log_var-K.square(z_mean)-K.exp(z_log_var),axis=-1)
    return K.mean(xent_loss+kl_loss)
  
  def call(self,inputs):
    x = inputs[0]
    z_decoded = inputs[1]
    loss = self.vae_loss(x,z_decoded)
    self.add_loss(loss,inputs=inputs)
    return x 
  
y = CustomVariationLayer()([input_qus,z_decoded])

NameError: ignored

In [0]:
vae = Model(input_qus,y)

vae.compile(optimizer='rmsprop',loss=None)
vae.summary()

vae.fit()